In [352]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from datetime import date
from copy import deepcopy
import json

# Import dataset
aurora = pd.read_csv("customers (3).csv")

In [353]:
# Extraer las variables de enganche
aurora_raw = aurora.drop(columns=['@01_lactante' , '@01_lactante_plural' , '@01_red_v' ,'@01_vocacion',
                                  '@01_vocacion_opt', '@01_vocacion_otra', '@01_vocacion_text', '@02_recursos_v', '@02_vocacion',
	                              '@02_vocacion_opt' ,'@02_vocacion_otra', '@02alim_lactante', '@03_amamantar', '@03_informacion_v',	
                                  '@03_vocacion',	'@03_vocacion_opt',	'@03_vocacion_pais',	'@04_razones_v', '@04_razones_v_otro',	
                                  '@04_servicios_lactante',	'@04_vocacion',	'@04_vocacion_opt',	'@04_vocacion_pais', '@05_vacuna_lactante',	
                                  '@05_vocacion', '@05_vocacion_text', '@06_vocacion',	'@06necesidad_lactante', '@07_vocacion', '@07alim_nna',	
                                  '@08_vocacion', '@08servicios_hasta5', '@09vacuna_nna' , '@12servicios_nna' ,	'@13edu_nna' , '@14nivel_nna',
                                  '@10necesidad_hasta5', '@11alim_nna', '@3_wash', '@3_wash_otro', '@3_wash_text', '@4_salud', '@4_salud_otro',	
                                  '@4_salud_text', '@5_sectorotro',	'@5_sectorotro_name', '@5_sectorotro_text', '@6_offi_asist', '@6_offi_gestor',
                                  '@15docu_nna', '@16requisitos_nna', '@17vacuna_nna', '@18necesidad_nna', '@2_sector_offi', '@name', '@no_viaje',	
                                  '@no_viaje_otro',	'@no_viaje_text', '@pais_actual_monitoreo', '@opt_no_viaje', '@opt_paisl_monitoreo', '@opt_razones_v', 
                                  '@opt_salud', '@opt_sector', '@opt_servi_hasta5', '@opt_servi_lactante', '@opt_servi_nna', '@opt_ultimo_monitoreo',	
                                  '@opt_wash',	'@option_name',	'@pais_actual_monitoreo' ]).copy()

                                  




In [354]:
aurora_raw.head()
aurora_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Columns: 152 entries, @0_flow to @zona_pais
dtypes: bool(2), float64(26), int64(2), object(122)
memory usage: 386.3+ KB


In [355]:
# Rename variables according to Carto names (first round)

newColumns = {
    '@phone': 'phone',
    '@id': 'id',
    '@conexion_date': 'conexion_date',
    '@date_registred': 'date_registered',
    '@0_flow': 'flow_type',
    '@1_gestor': 'gestor',
    '@zona_pais': 'zona_pais',
    '@15_paisnac' : 'pais_nac',
    '' : '',
}

# there are still missing variables of dates, geographical coordinates, feeback of services.
aurora_raw = aurora_raw.rename(columns=newColumns)

In [356]:
# Eliminate observations of Aurora team phones.
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56994154407].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56963085715].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56930150973].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56956195946].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573245502367].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573002983157].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573044640378].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573133910738].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573185891917].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573182396683].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573206646548].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573216903526].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573105523263].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50685075883].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50660030436].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50763068014].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50768244084].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 51963852497].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50768689439].index)
print(aurora_raw.date_registered.info())

<class 'pandas.core.series.Series'>
Int64Index: 310 entries, 0 to 327
Series name: date_registered
Non-Null Count  Dtype 
--------------  ----- 
310 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB
None


In [357]:
# Replace Aurora v2.0 for Enganche, is the same category
aurora_raw['flow_type'] = aurora_raw['flow_type'].replace(to_replace="Aurora v2.0",
                                                          value="Enganche")
# Replace Paso canoas lower case, is the same category
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Paso canoas - Costa Rica",
                                                          value="Paso Canoas - Costa Rica")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="San Juan de Pasto - Colombia",
                                                          value="Ipiales - Colombia")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="los patios - Colombia",
                                                          value="Los patios - Colombia")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Colchane - Bolivia",
                                                          value="Colchane - Chile")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Tacna - Perú",
                                                          value="Arica - Chile")


# Replace bad categories in gestor
aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Viviana Castro",
       value="9")
#aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Rosa Casanova",
        #   value="7")
aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Deissy Rodríguez",
          value="10")
#aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Luis David Calderón",
          # value="8")
aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="José Luis Gaete",
           value="3")
#aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Glenis Barragán",
          # value="6")
aurora_raw['gestor'] = aurora_raw['gestor'].replace(to_replace="El 5",
                                                    value="5")

In [358]:
aurora_raw["date_registered"] = pd.to_datetime(
    aurora_raw["date_registered"], format='%d-%m-%Y %H:%M:%S')
aurora_raw["date_"] = aurora_raw["date_registered"].dt.strftime("%Y-%m-%d")

In [359]:
# Use dictionary to give other values to the variable
gestor_dict = {
    1: "Glenis Barragán",
    2: "José Herrera",
    3: "José Luis Gaete",
    4: "Luis David Calderón",
    5: "Luisa Solis",
    6: "Marianela Varela",
    7: "Massiel Rodriguez",
    8: "Rosa Casanova",
    9: "Sheila Muñoz",
    10: "Valentina Medina",
    11: "Viviana Castro",
    12: "Yuliana Chiquillo",
}

def gestorMap(value):
    try:

        if (type(value) == int):
            return gestor_dict[value]
        if (type(value) == str):
            return gestor_dict[int(value)]
    except Exception as e:
        return value


aurora_raw['gestor'] = aurora_raw['gestor'].apply(lambda x: gestorMap(x))

In [351]:
aurora_raw['pais_nac'].value_counts()
df['veh'].value_counts()[df['veh'].value_counts() > 2]

Venezuela      195
Colombia        23
Ecuador          7
Bolivia          7
Chile            4
Perú             2
Haití            2
Colombia 🇨🇴      2
Otro             1
Name: pais_nac, dtype: int64

In [338]:
unpivoted = aurora_raw.groupby(['zona_pais', 'flow_type', 'date_'])[
    'id'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='date_',
    index=['zona_pais', 'flow_type'],
    values='id')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()
pivoted

date_,zona_pais,flow_type,2023-10-04,2023-10-11,2023-10-12,2023-10-13,2023-10-14,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-23,2023-10-24,2023-10-25
0,Arica - Chile,Enganche,1.0,3.0,NaN,5.0,2.0,2.0,1.0,4.0,NaN,6.0,NaN,2.0,1.0,NaN
1,Arica - Chile,Monitoreo-Feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN
2,Arica - Chile,Monitoreo-NNA,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN
3,Arica - Chile,Monitoreo-Percepción,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN
4,Arica - Chile,Monitoreo-vocación,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
5,Arica - Chile,QR-Enganche,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,Bochalema - Colombia,Enganche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
7,Bochalema - Colombia,QR-Enganche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,Colchane - Chile,Enganche,NaN,NaN,NaN,NaN,NaN,NaN,8.0,7.0,5.0,3.0,NaN,NaN,NaN,NaN
9,Colchane - Chile,Monitoreo-Feedback,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [294]:
unpivoted = aurora_raw.groupby(['zona_pais', 'gestor', 'date_'])[
    'id'].count().reset_index()
pivoted1 = unpivoted.pivot(
    columns='date_',
    index=['zona_pais', 'gestor'],
    values='id')
# Reset the index of the pivoted DataFrame
pivoted1 = pivoted1.reset_index()
pivoted1

date_,zona_pais,gestor,2023-10-11,2023-10-12,2023-10-13,2023-10-14,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-23,2023-10-24
0,Arica - Chile,Deissy Rodríguez,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arica - Chile,Luis David Calderón,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arica - Chile,Luisa Solis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
3,Arica - Chile,Rosa Casanova,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arica - Chile,Sheila Muñoz,NaN,NaN,NaN,NaN,2.0,NaN,8.0,NaN,6.0,NaN,2.0,NaN
5,Arica - Chile,Valentina Medina,NaN,NaN,NaN,NaN,2.0,1.0,2.0,NaN,3.0,NaN,1.0,1.0
6,Arica - Chile,Viviana Castro,1.0,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Bochalema - Colombia,Glenis Barragán,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
8,Colchane - Chile,Luisa Solis,NaN,NaN,NaN,NaN,1.0,11.0,10.0,7.0,4.0,NaN,NaN,NaN
9,Corredor - Costa Rica,Marianela Varela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN


In [325]:

df1 = aurora_raw.groupby('@14_intencion').count()
df1

,flow_type,@12_dias,@14_intencionotro,pais_nac,@15_paisnacotro,@16_paisinicio,@16_paisinicio_otro,@17_paisvivi,@17_paisvivi_otro,@18_alimentos,...,@ubicacion,@ultima_otro,@ultima_zona_pais,@ultimo_pais,@ultimo_pais_otro,@wash_temp,@zona,@zona_monitoreo,@zona_otro,zona_pais
@14_intencion,,,,,,,,,,,,,,,,,,,,,
1,50,54,0,54,0,54,0,54,1,54,...,31,9,16,14,2,0,1,7,3,54
2,2,2,0,2,0,2,0,2,0,2,...,1,0,0,0,0,0,0,0,0,2
3,6,6,5,5,0,5,0,5,0,5,...,4,0,0,0,0,0,0,1,0,6
4,2,2,0,2,0,2,0,2,1,2,...,1,0,0,0,0,0,0,1,0,2
Tengo intenciones de establecerme,10,11,0,10,0,10,0,10,0,10,...,10,0,3,3,0,0,0,0,0,11


In [360]:
excel_file = 'aurora.xlsx'
aurora_raw.to_excel(excel_file, index=False)